In [1]:
import os
import shutil
from glob import glob
import cv2

def rearrange_files(source_folder, destination_folder):
    for root, dirs, files in os.walk(source_folder):
        relative_path = os.path.relpath(root, source_folder)
        destination_path = os.path.join(destination_folder, relative_path)
        os.makedirs(destination_path, exist_ok=True)

def move_files(source_folder):
    destination_folder = source_folder
    for root, dirs, files in os.walk(source_folder):
        relative_path = os.path.relpath(root, source_folder)
        destination_path = os.path.join(destination_folder, relative_path)
        os.makedirs(destination_path, exist_ok=True)
        for file in files:
            source_file = os.path.join(root, file)
            destination_file = os.path.join(destination_path, file)
            shutil.move(source_file, destination_file)
            print(f'Moved {source_file} to {destination_file}')
            
def move_files(source_folder, destination_folder):
    for root, dirs, files in os.walk(source_folder):
        relative_path = os.path.relpath(root, source_folder)
        destination_path = os.path.join(destination_folder, relative_path)
        os.makedirs(destination_path, exist_ok=True)
#source_folder = 'source_folder'


In [2]:
reference_folder = "/mnt/atlas/data_KPIs/data/KPIs24_Training_Data/Task1_patch_level/split_small_patches_by_class_updated/"
source_folder = "/mnt/atlas/data_KPIs/data/KPIs24_Training_Data/Task1_patch_level/pix2pix_data_augmentation/large_patches/"
destination_folder = "/mnt/atlas/data_KPIs/data/KPIs24_Training_Data/Task1_patch_level/pix2pix_data_augmentation_rearranged/"


In [3]:
# #add information about what pix2pix model was used to generate the images (one model for each class)
# for root, dirs, files in os.walk(source_folder):
#     relative_path = os.path.relpath(root, source_folder)
#     for file in files:
#         # print(os.path.join(root, file), os.path.join(root, 'pix2pix_' + root.split('/')[-2] + '_' +  file))
#         shutil.move(os.path.join(root, file), os.path.join(root, 'pix2pix_' + root.split('/')[-2] + '_' +  file))

In [4]:
rearrange_files(reference_folder, destination_folder)

In [5]:
images = sorted(glob(os.path.join(source_folder, "**/*img*"), recursive=True)) #sorted(glob(os.path.join(datadir, "**/*img.jpg"), recursive=True))
masks = sorted(glob(os.path.join(source_folder, "**/*mask*"), recursive = True)) #sorted(glob(os.path.join(datadir, "**/*mask.jpg"), recursive = True))

In [6]:
#remove all elements that are not files (directories)
images = [x for x in images if os.path.isfile(x)]
masks = [x for x in masks if os.path.isfile(x)]

In [7]:
img_case_id_folders = [root for root, dirs, files in os.walk(destination_folder) if 'img' in root]
mask_case_id_folders = [root for root, dirs, files in os.walk(destination_folder) if 'mask' in root]

In [8]:
# for folder in img_case_id_folders:
#     #move images to destination folder 
#     for img in images:
#         #check if name file is in the format X-X instead of X_X
#         a = img.split('/')[-1].split('-')
#         if len(a) > 1:
#             # print(a)
#             img_name = a[0] +'_'+ a[1].split('_')[0]
#             # print(img_name, folder)
#             if img_name in folder:
#                 print(img_name, folder, folder.split('/')[-2], img_name, folder.split('/')[-3], img)
#                 # shutil.move(img, folder)
#                 print(f'move {img} to {folder}')
#                 #copy 
#                 print('copy')
#                 # shutil.copy(img, folder)

In [9]:
# for folder in img_case_id_folders:
#     #move images to destination folder 
#     for img in images:
#         #check if name file is in the format X-X instead of X_X
#         a = img.split('/')[-1].split('-')
#         if len(a) > 1:
#             img_name = a[0] +'_'+ a[1].split('_')[0]
#         else:
#             img_name = a[0]
        
#         if folder.split('/')[-2] in img_name:
#             print(f'move {img} to {folder}')
#             shutil.copy(img, folder)

for folder in mask_case_id_folders:
    #move masks to destination folder 
    for mask in masks:
        #check if name file is in the format X-X instead of X_X
        a = mask.split('/')[-1].split('-')
        if len(a) > 1:
            mask_name = a[0] +'_'+ a[1] # a[0] +'_'+ a[1].split('_')[0]
        else:
            mask_name = a[0]
    
        if folder.split('/')[-2] in mask_name:
            print(f'move {mask} to {folder}')
            # shutil.copy(mask, folder)
            mask_array = cv2.imread(mask)
            mask_array = cv2.cvtColor(mask_array, cv2.COLOR_RGB2GRAY)
            cv2.imwrite(os.path.join(folder, mask_name), mask_array)

move /mnt/atlas/data_KPIs/data/KPIs24_Training_Data/Task1_patch_level/pix2pix_data_augmentation/large_patches/56Nx/mask/pix2pix_56Nx_08_474_01_100_12288_4096_mask.png to /mnt/atlas/data_KPIs/data/KPIs24_Training_Data/Task1_patch_level/pix2pix_data_augmentation_rearranged/NEP25/08_474_01/mask
move /mnt/atlas/data_KPIs/data/KPIs24_Training_Data/Task1_patch_level/pix2pix_data_augmentation/large_patches/56Nx/mask/pix2pix_56Nx_08_474_01_101_13312_4096_mask.png to /mnt/atlas/data_KPIs/data/KPIs24_Training_Data/Task1_patch_level/pix2pix_data_augmentation_rearranged/NEP25/08_474_01/mask
move /mnt/atlas/data_KPIs/data/KPIs24_Training_Data/Task1_patch_level/pix2pix_data_augmentation/large_patches/56Nx/mask/pix2pix_56Nx_08_474_01_103_15360_4096_mask.png to /mnt/atlas/data_KPIs/data/KPIs24_Training_Data/Task1_patch_level/pix2pix_data_augmentation_rearranged/NEP25/08_474_01/mask
move /mnt/atlas/data_KPIs/data/KPIs24_Training_Data/Task1_patch_level/pix2pix_data_augmentation/large_patches/56Nx/mask/p

In [10]:
mask_name

'pix2pix_56Nx_normal_F4_9_9216_0_mask.png'

In [11]:
# for img, mask in zip(images, masks):
#     img_name = os.path.basename(img)
#     mask_name = os.path.basename(mask)
#     img_class = img.split('/')[-3]
#     for root, dirs, files in os.walk(destination_folder):
#         if img_class in root:
#             if root.split('/')[-1] == 'img' and root.split('/')[-2] in img_name:
#                 print(f'copy {img} to {root}')

In [12]:
a[1]

IndexError: list index out of range

In [ ]:
a[0] +'_'+ a[1]

In [ ]:
os.path.join(folder, mask_name)

In [ ]:
# #cp files from source to destination folder in the correct subfolder based on file names
# for root, dirs, files in os.walk(source_folder):
    
#     for file in files:
#         if file.endswith('.png'):
#             source_file = os.path.join(root, file)
#             destination_file = os.path.join(destination_folder, file)
#             # shutil.copy(source_file, destination_file)
#             print(f'Copied {source_file} to {destination_file}')
#             #move_files(source_folder, destination_folder)
#             #shutil.move(source_file, destination_file)
#             #print(f'Moved {source_file} to {destination_file}')